# Projekt 5 - Távirányítású autó

A XXI. században az autóipár újra fénykorát éli, főleg az elektromos autók újbóli megjelenésével. A technológia már akkorát fejlődött, hogy egyes cégek már elkezdték tesztelni az önvezető autókat is. Minden jel arra mutat, hogy a jövőben a volán mögött ülve nem kell majd az utat figyelnünk, mert megteszi azt helyettünk az autónk. 

Ilyen autókat létrehozni nem egyszerű és sok munka/kutatás áll mögötte. Ha időben és megfelelő részletességgel kezdjük el tanulmányozni ezeket a technológiákat, előbb utóbb nem csak szemlélői hanem alakítói is lehetünk ennek a fejlődésnek. 

Ebben a projektben az alapokat próbáljuk meg elsajátítani. Egy két kerék meghajtású kis járművet rakunk össze, amit a számítógépünk billentyűzetének segítségével vezérlünk. Lényegében egy távirányítású elektromos autót szerelünk össze és programozzuk be működésre.

## Mit fogsz készíteni?

Kettő 3-6V DC Motorból és egy L293D vezérlő (driver) chipből álló rendszert építünk, amit a Raspberry Pi-hoz kapcsolunk. A két motor vezérlését a billentyűzetről oldjuk meg, méghozzá úgy, hogy VNC-n keresztül jelentkezünk be a Raspberry Pi-ra, így az autó mozgását csak a tápkábel korlátozza. A nyílakkal ellátott billentyűk irányítják majd az aztót előre, hátra, jobbra és balra. Lényegében egy saját gyártású távirányítós autót készítünk. **Megjegyzés: Mindig használjunk megfelelő drivert a motorral összekötve, hogy csökkentsük bármelyik eszköz károsodásának lehetőségét.**

## Mit tanulsz meg?

A dc motoros autó elkészítésével a következőket tanulod meg:

* Hogyan tudsz kettő DC motort egy áramkörbe beiktatni a hozzá tartozó driverrel együtt.
* Hogyan programozd be a Raspberry Pi GPIO kimeneteleit a **gpiozero** modullal.
* Hogyan hozzunk létre függvényeket a mozgásirányok leszimulálására.
* Hogyan használjuk a pygame csomagot a billentyűzet figyelésére.
* Hogyan mozgassuk és állítsuk le a motorokat VNC kapcsolaton keresztül.

## A projekt részletekre bontása

* Elkészíteni az áramkört.
* Inicializálni a Motor objektumokat.
* Definiálni egy-egy függvényt a jármű előre, hátra, jobbra és balra mozgatására.
* Figyelni a billentyűzetet és a megfelelő billentyűk lenyomásakor a megfelelő mozgást végrehajtani.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) 2db DC Motor: [itt vásárolhatsz](https://www.tme.eu/hu/details/oky5022-1/dc-motorok/okystar/)

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

d) L293D vezérlő: [itt vásárolhatsz](https://www.tme.eu/hu/details/l293d/motor-es-pwm-driverek/stmicroelectronics/)

e) 2db nagy kerék [https://www.tme.eu/hu/details/df-fit0003/robotika-es-rc-kellekek/dfrobot/fit0003/](https://www.tme.eu/hu/details/df-fit0003/robotika-es-rc-kellekek/dfrobot/fit0003/)

f) 1db kis kerék [https://www.tme.eu/hu/details/hy006-01003/robotika-es-rc-kellekek/emax/emx-ac-1353/](https://www.tme.eu/hu/details/hy006-01003/robotika-es-rc-kellekek/emax/emx-ac-1353/) vagy [https://www.tme.eu/hu/details/pololu-950/robotika-es-rc-kellekek/pololu/ball-caster-with-3-8-plastic-ball/](https://www.tme.eu/hu/details/pololu-950/robotika-es-rc-kellekek/pololu/ball-caster-with-3-8-plastic-ball/)

g) 1db doboz 

h) Szigetelő szalag

i) 1db kb. 5-7 cm hosszú 1mm vastag drót vagy gemkapocs darab

## A kapcsolási rajz

<img src="prog04_schema_noButtons.png" width=600 height=400 />

### Kapcsolási rajz részletező

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t. Az áramkör részletes összekötési magyarázata a [Motorok]() című bevezető leírásban található.

1) Az L293D driver földelés lábait kössük össze egymással és az átellenes oldallal (fekete drót). Egyben az áramforrás (4 db AAA elem a rajzon) negatív végét és a Raspberry Pi egyik földelését is kössük össze az L293D driver földelésével. Így minden elem azonos földelésen lesz.

2) Az áramforrás pozitív végét kapcsoljuk a driver *VS* lábára (*8*-as láb, piros drót). Ez adja a feszültséget a motor meghajtásához. 

3) Kapcsoljuk a Raspberry Pi +5V-os tüskéjét a driver *VSS* (*16*-os) lábával össze (fehér drót). Ez adja a driver működéséhez nélkülözhetetlen tápot. 

4) A Raspberry Pi *GPIO25*-ös tüskéjét kapcsoljuk az *EN1* (*1*-es) lábra (okkersárga drót). A driver ezen lába szolgál arra, hogy aktiválja a driver *IN1 (2), IN2 (7), OUT1 (3) és OUT2 (6)* lábait, amennyiben magas állapotba kerül. Alacsony állapotból az előbb említett lábak nem kapnak áramot. 

5) A Raspberry Pi *GPIO11*-es tüskéjét kapcsoljuk az *EN2* (*9*-es) lábra (okkersárga drót). A driver ezen lába szolgál arra, hogy aktiválja a driver *IN3 (10), IN4 (15), OUT3 (11) és OUT4 (14)* lábait, amennyiben magas állapotba kerül. Alacsony állapotból az előbb említett lábak nem kapnak áramot. Így két motort is tudunk már vezérelni.

6) A Raspberry Pi *GPIO23*-as és *GPIO24*-es tüskéit kapcsoljuk az *IN1 (2), IN2 (7)* lábakra a driveren (szürke és narancssárga drótok). Felváltva aktiválva őket vagyunk képesek előre és hátrafele forgatni a motort attől függően, hogy épp melyik van magas állapotban. 

7) A Raspberry Pi *GPIO9*-es és *GPIO10*-es tüskéit kapcsoljuk az *IN3 (10), IN4 (15)* lábakra a driveren (szürke és narancssárga drótok). Felváltva aktiválva őket vagyunk képesek előre és hátrafele forgatni a második motort attől függően, hogy épp melyik van magas állapotban.

8) A motor két kimenetét (citromsárga és zöld drótok) kapcsoljuk a driver *OUT1 (3) és OUT2 (6)* lábaira. Az *IN1 és IN2* lábak állapota szabályozza, hogy ezek a lábak, azaz maga a motor, kap-e feszültséget.

9) A második motor két kimenetét (citromsárga és zöld drótok) kapcsoljuk a driver *OUT3 (11) és OUT4 (14)* lábaira. Az *IN3 és IN4* lábak állapota szabályozza, hogy ezek a lábak, azaz maga a motor, kap-e feszültséget.

**Kétszer is ellenőrizzük le, hogy a bekötésünk rendben van-e. A félrekötött tüskék nagyban növelik a motor vagy a Raspberry Pi tökremenési esélyeit. A motort SEMMIKÉPP SE tápláljuk és irányítsük direktben a Raspberry Pi-ről, mert az szinte biztosan a számítógép sérükéséhez vezet.**

### Projekt összeszerelési útmutató

Az autó karosszériáját bármilyen terv alapján elkészíthetjük, az itt bemutatott leírás inkább egy prototípusnak felelne meg. Ha komolyabb karosszérát szeretnénk, akár 3D-s nyomtatóval is készíthetünk a projekthez. 

<img src="dc_car_1.JPG" width=600 height=400 />

A távvezérelt autónk összeszereléséhez a fenti képen látható elemek fognak kelleni. Egy nagy doboz, amibe beférnek a motorok, a Raspberry Pi, az AA elemek és a breadboard. Szükség lesz két nagy méretű kerékre, ami illeszkedik a motorok tengelyére. Ezek adják az autó meghajtását. Emellett kell egy kisebb kerék is, ami elsőkerékként fog szolgálni.

<img src="dc_car_2.JPG" width=600 height=400 />

Először vágjunk lyukat a doboz oldalsó falain, hogy a motor tengelye kiférjen rajta.

<img src="dc_car_3.JPG" width=600 height=400 />

Ezután illesszük be a motorok tengelyeit a lyukba és kívülről rögzítsük a tengelyre a nagy kereket. Hogy a motor ne forogjon a dobozban, szigetelőszallaggal lerögzíthetjük a dobozhoz. 

<img src="dc_car_4.JPG" width=600 height=400 />

Az első kerék felszereléséhez vágjunk függőleges irányba lyukat a dobozba úgy, hogy a kisebb kerék kényelmesen beférjen. Majd a rövid drótszerű tengelyt dugjuk át a kerék középpontjában levő lyukon és rögzítsük a drótot a dobozhoz. 

<img src="dc_car_last.JPG" width=600 height=400 />

Innentől kezdve már csak össze kell kötni a drótokat az áramkörben és a jármű indulásra készen áll. 

## A kód
Nyissunk meg egy új python fájlt és mentsük el pl. ```dcmotor_car.py``` név alatt.

### A DC motor kipróbálása

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, a motor mozgatására.

Először betöltjük a ```gpiozero``` modulból a ```Motor``` objektumot ami lehetővé teszik, hogy a Raspi kommunikáljon a motorral a hozzá tartozó vezérlővel a **GPIO** tüskéken keresztül. Emellett beimportáljuk a

* ```time``` függvényt is, hogy szünetet tarthassunk a python kód lefutásában,
* ```sys``` csomagot, hogy ki tudjunk lépni a pythonból,
* ```pygame, pygame.freetype``` csomagokat, hogy tudjuk figyelni melyik billentyűt nyomták le.

```dcmotor_car.py```:

In [ ]:
from gpiozero import Motor
import sys, pygame, pygame.freetype
import time

Az objektum beolvasása után inicializálunk kettő ```Motor``` objektumot, amit ```motor_left```-nek és ```motor_right```-nak nevezünk el. A bal oldali motort a *GPIO25* tüskével aktiváljuk és a *GPIO23* és *GPIO24* tüskék felelnek a motor előre és hátra forgatásáért. A jobb oldali motort a *GPIO11* tüskével aktiváljuk és a *GPIO10* és *GPIO09* tüskék felelnek a motor előre és hátra forgatásáért. 

```dcmotor_car.py```:

In [ ]:
from gpiozero import Motor, Button
import sys, pygame, pygame.freetype
import time

motor_left = Motor(forward=23, backward = 24, enable=25)  
motor_right = Motor(forward=10, backward = 9, enable=11)

Ezután készen állunk arra, hogy leteszteljük a motorok mozgatását és az objektumhoz tartozó egyéb metódusok működését. Elsőnek a bal motort előre forgatjuk ```motor_left.forward()```, 2 másodpercen keresztül, majd megállítjuk a ```motor_left.stop()``` paranccsal és leteszteljük, hogy hátrafele is tud-e forogni, ```motor_left.backward()```. Hagyjuk forogni 2 másodpercet, majd újra leállítjuk. Ezután a jobb oldali motort teszteljük a ```motor_right.forward()```, ```motor_right.stop()``` és a ```motor_right.backward()``` parancsokkal.

```dcmotor_car.py```:

In [ ]:
from gpiozero import Motor, Button
import sys, pygame, pygame.freetype
import time

motor_left = Motor(forward=23, backward = 24, enable=25)  
motor_right = Motor(forward=10, backward = 9, enable=11)

motor_left.forward()
sleep(2)
motor_left.stop()

motor_left.backward()
sleep(2)
motor_left.stop()

motor_right.forward()
sleep(2)
motor_right.stop()

motor_right.backward()
sleep(2)
motor_right.stop()

Ha lefuttatjuk ezt a kódot, kiderül, hogy mindent rendben kötöttünk-e össze az áramkörben. Ha nem az elvárt mozgásokat tapasztaljuk, akkor meg kell keresni a hibát. Nagy valószínűséggel az áramköri kapcsolásokkal van gond.

### A pygame képernyő inicializálása

A számítógép billentyűzet lenyomásának detektálásának egyik legegszerűbb módja a *pygame* csomag használata. Ahogy említettük már egy korábbi projektben, ez a csomag a játékok létrehozását könnyíti meg pythonban. Egyik fontos funkciója a bemeneti jelek észlelése mint pl. az egér mozgatása vagy a billentyűzet nyomkodása. 

Először inicializálni kell a *pygame* környezetet a ```pygame.init()``` paranccsal. Evvel lehetővé válik egy kijelző felület megjelenítése, ahol a játékunk jelenne meg, de mi csak megjelenítjük és nem használjuk. Ha aktív az így létrehozott felület, akkor képes a bemeneti jeleket észlelni. A kijelző felület létrehozásához meg kell adnunk a méretét egy *tuple* változó formájában, aminek első értéke a szélesség a második a magasság, ```size = (500, 500)```. A méretünk mértékegysége a pixel. Ezután már létrehozhatjuk a kijelző felületet a ```pygame.display.set_mode(size)```, paranccsal és a felületet a ```screen``` változóban tároljuk. 

Hogy leteszteljük a felületünk működését és hogy valóban érzékeli a billentyüzetünket, elindítunk egy végtelen ciklust, amiben figyeljük történt-e valamilyen esemény. Az eseményeket a ```pygame.event.get()``` parancs figyeli és gyűjti magába. Egy ```for``` ciklus segítségével lépkedünk végig az érzékelt eseményeken (feltéve, hogy épp bekövetkezett valami), ```for event in pygame.event.get():```. Ha észleltünk valamilyen eseményt, akkor azt kinyomtatjuk a képernyőre, ```print(event)```, hogy meggyőződhessünk, valóban a jó eseményt észleljük. Végül a ```time.sleep(0.1)``` paranccsal késleltetjük a program futását, hogy legyen ideje feldolgozni a különböző eseményeket. 

```dcmotor_car.py```:

In [ ]:
from gpiozero import Motor
import sys, pygame, pygame.freetype
import time

motor_left = Motor(forward=23, backward = 24, enable=25)  #
motor_right = Motor(forward=10, backward = 9, enable=11)

pygame.init( )

#Set canvas parameters
size = 500, 500

#Set size of display
screen = pygame.display.set_mode(size)

while 1:
    for event in pygame.event.get():
        print(event)
    time.sleep(0.1)#Wait for 100ms before next button press

A fenti kódot lefuttatva egy fekete hátterű ablak jelenik meg a képernyőn, és minden egyes egér mozgatásnál vagy billentyű lenyomásánál, kiírja a képernyőre az egér pozícióját vagy hogy melyik billentyűt nyomtuk meg. 

### A nyíl jelű billentyűzetek érzékelése pygame-en keresztül

Ahogy már említettük, a nyíl jelű billentyűkkel tervezzük a jármű mozgatását. Ebben a fejezetben lekódolunk egy teszt billentyűzet észlelést.

Az előző fejezetben már láttuk, hogyan kell észlelni az eseményeket, ```event```. Ezeknek vannak tulajdonságaik, amikre rá lehet kérdezni. Elsőként, megkérdezzük, hogy az esemény típusa, ```event.type```, megegyezik-e az ablak bezárásával (piros X-re kattintás), ```pygame.Quit```. Ha igen, akkor bezáratjuk az ablakot és kilépünk a pythonból, ```sys.exit()```. 

Következő dolog amit leellenőrzünk, hogy billentyüzet lenyomás történt-e, azaz ```if event.type == pygame.KEYDOWN:```. Ha igen, akkor kivizsgáljuk, hogy a nyíl jelű gombokat nyomták-e le, pl. ```if event.key == pygame.K_RIGHT:```. Az ```event.key``` tartalmazza a billentyűt, amit lenyomtak, míg a ```pygame.K_RIGHT``` a jobbra nyílú billentyűnek felel meg. Ha ezt valóban lenyomták, akkor kinyomtatjuk a képernyőre, hogy mozágs jobbra, ```print('Move right...')```. Ugyanígy leellenőrizzük a balra, ```pygame.K_LEFT```, le ```pygame.K_DOWN```és fel ```pygame.K_UP``` billentyűket is és a hozzájuk tartozó mozgást nyomtatjuk ki a képernyőre. 

Végül, a motornak addig kellene forognia, amíg a billentyűt lenyomva tartjuk, így a billentyű felengedésével ki kell nyomtatnunk a képernyőre, hogy megállt a motor. Tehát azt is le kell ellenőriznünk, hogy történt-e billentyű felengedés, ```if event.type == pygame.KEYUP:```. Ha igen, akkor kiírjuk, hogy a motorok leálltak. 

```dcmotor_car.py```:

In [ ]:
from gpiozero import Motor
import sys, pygame, pygame.freetype
import time

motor_left = Motor(forward=23, backward = 24, enable=25)  #
motor_right = Motor(forward=10, backward = 9, enable=11)

pygame.init( )

#Set canvas parameters
size = 500, 500

#Set size of display
screen = pygame.display.set_mode(size)

while 1:
    for event in pygame.event.get():
        # print(event)
        if event.type == pygame.QUIT:
            sys.exit()
            
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RIGHT:
                print('Move right...')

            if event.key == pygame.K_LEFT:
                print('Move left...')

            if event.key == pygame.K_DOWN:
                print('Move backward...')

            if event.key == pygame.K_UP:
                print('Move forward...')

        if event.type == pygame.KEYUP:
            print('Stop motors...')
            
    time.sleep(0.1)#Wait for 100ms before next button press

Lefuttatva ezt a kódot és nyomkodva a nyíl jelű billentyűket, a képernyőn meg kell jelennie a szövegeknek, jelezve, hogy a mozgás melyik irányba történt.

### Kerék forgató függvények definiálása

Ahhoz, hogy mozgásra bírjuk a kis járművünket a motorok forgatását össze kell hangolni. Azaz, mindegyik mozgási irányhoz meg kell határoznunk, hogy melyik motort merre kell forgatni. Ezt főként megfigyeléssel lehet elérni és a szerkezet összeszerelése után is érdemes még egyszer leellenőrízni, hogy a motorok a megfelelő irányba forognak, létrehozva a kívánt mozgást, nem pedig akadályoztatják egymást, megbénítva így a járművet.

Öt mozgásfüggvényt definiálunk. Az utolsó és talán a legegyszerűbb a motorok leállítása, ```stop_motors(m1, m2)```. Itt bemenő paraméterként megadjuk mindkét motort és a függvény belsejében leállítjuk őket a ```stop()``` metódussal. A másik négy függvény az előre, hátra, jobbra és balra mozgásirányokat definiálják. Mindegyiknek három bemenő paramétere van, a két motor objektuma és a sebesség paraméter (mindkét motort ugyanakkora sebességgel mozgatjuk). 

A mi kódunkban (de ahogy előbb említettük, ezt mindenki maga tesztelje), az előre mozgást, ```move_forward(m1, m2, speed=0.8)```, úgy érjük el, hogy mindkét motort előre forgatjuk. A hátrafele mozgáshoz, ```move_backward(m1, m2, speed=0.8)```, mindkét motort hátrafele forgatjuk. Balra mozgáshoz, ```move_left(mleft, mright, speed=0.8)```, a bal motort hátra mozgatjuk (és ehhez lehetne még a jobb oldali motort előre mozgatni). Jobbra mozgáshoz, ```move_right(mleft, mright, speed=0.8)```, pedig pont az ellenkezőt csináljuk mint az előbb, a jobb oldali motort hátrafele mozgatjuk (és akár a bal oldali motort is lehet előre mozgatni). 

In [1]:
def move_forward(m1, m2, speed=0.8):
    m1.forward(speed=speed)
    m2.forward(speed=speed)

def move_backward(m1, m2, speed=0.8):
    m1.backward(speed=speed)
    m2.backward(speed=speed)

def move_left(mleft, mright, speed=0.8):
    mleft.backward(speed=speed)
    #mright.forward(speed=speed)

def move_right(mleft, mright, speed=0.8):
    #mleft.forward(speed=speed)
    mright.backward(speed=speed)

def stop_motors(m1, m2):
    m1.stop()
    m2.stop()


Mégegyszer kihangsúlyozzuk, hogy a mozgásirányok definiálásához az aktuális összeszerelést kell figyelembe venni és letesztelni még mielőtt terhelés kerülne az autóra. Ha apró zúgás hallatszódik a motorokból, de érdemleges mozgás nincs, ellenőrízzük, hogy a motor mennyire van terhelve, illetve, hogy jó irányba vannak-e definiálva a motorok forgása. 

### Mozgató függvények meghívása billentyűzettel

Most már rendelkezésünkre áll minden építő elem, hogy összerakjuk a járműhöz a vezérlő kódot. Bemásolva a mozgató függvényeket és a billentyűzet ellenőrzésénél, ha teljesül a feltétel meghívjuk ezeket a függvényeket. Például a jobbra billentyű lenyomásánál, ```if event.key == pygame.K_RIGHT:``` meghívjuk a ```move_right(motor_left, motor_right, speed=1)``` mozgató függvényt, majd ha elengedtük a billentyűt, ```f event.type == pygame.KEYUP:```, akkor a ```stop_motors(motor_left, motor_right)``` függvényt hívjuk meg.

```dcmotor_car.py```:

In [ ]:
from gpiozero import Motor
import sys, pygame, pygame.freetype
import time


motor_left = Motor(forward=23, backward = 24, enable=25)  #
motor_right = Motor(forward=10, backward = 9, enable=11)


pygame.init( )
#Set canvas parameters
size = 500, 500
#Set size of display
screen = pygame.display.set_mode(size)


def move_forward(m1, m2, speed=0.8):
    m1.forward(speed=speed)
    m2.forward(speed=speed)

def move_backward(m1, m2, speed=0.8):
    m1.backward(speed=speed)
    m2.backward(speed=speed)

def move_left(mleft, mright, speed=0.8):
    mleft.backward(speed=speed)
    #mright.forward(speed=speed)

def move_right(mleft, mright, speed=0.8):
    #mleft.forward(speed=speed)
    mright.backward(speed=speed)

def stop_motors(m1, m2):
    m1.stop()
    m2.stop()
    
    
while 1:
    for event in pygame.event.get():
        # print(event)
        if event.type == pygame.QUIT:
            sys.exit()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RIGHT:
                print('Move right...')
                move_right(motor_left, motor_right, speed=1)
            if event.key == pygame.K_LEFT:
                print('Move left...')
                move_left(motor_left, motor_right, speed=1)
            if event.key == pygame.K_DOWN:
                print('Move backward...')
                move_backward(motor_left, motor_right, speed=1)
            if event.key == pygame.K_UP:
                print('Move forward...')
                move_forward(motor_left, motor_right, speed=1)

        if event.type == pygame.KEYUP:
            print('Stop motors...')
            stop_motors(motor_left, motor_right)

    time.sleep(0.1)#Wait for 100ms before next button press

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```dcmotor_car.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```dcmotor_car.py```-t elmentettük. Ott begépelve a ```python dcmotor_car.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a program elindítása után megjelenik egy kis ablak a képernyőn és a gép várja a nyíllal megjelölt gombok lenyomását a billentyűzeten. Az autónak a nyílak irányának megfelelően kell elmozdulnia lenyomáskor, majd elengedéskor a motorokat le kell állítsa. 

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Írjuk át a kódot és tervezzük át az áramkört úgy, hogy ne a billentyűzetről, hanem gombokról kapja az irányítást.
* Tervezzük át az áramkört, hogy 4 LED diódával jelezzük a mozgás irányát a gombok lenyomásakor.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero leírás - [https://gpiozero.readthedocs.io/en/stable/recipes.html](https://gpiozero.readthedocs.io/en/stable/recipes.html)

2) Motor objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_output.html#motor](https://gpiozero.readthedocs.io/en/stable/api_output.html#motor)

3) L293D driver leírása - [https://www.ti.com/lit/ds/symlink/l293.pdf](https://www.ti.com/lit/ds/symlink/l293.pdf)

4) Pygame leírása - [https://www.pygame.org/news](https://www.pygame.org/news)

5) Pygame event - [https://www.raspberry-pi-geek.com/Archive/2014/05/Pygame-modules-for-interactive-programs](https://www.raspberry-pi-geek.com/Archive/2014/05/Pygame-modules-for-interactive-programs)